In [ ]:
!pip install gurobipy>=9.5.1
from gurobipy import *
e = Env(empty = True)
e.setParam('WLSACCESSID', '')
e.setParam('WLSSECRET','')
e.setParam('LICENSEID',)
e.start()

<gurobipy.Env, Parameter changes: WLSAccessID=(user-defined), WLSSecret=(user-defined), LicenseID=942593, OutputFlag=0>

In [ ]:
import random
import copy
import networkx as nx
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import math
import pickle
import os
from datetime import datetime
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
random.seed(datetime.now().timestamp())

In [ ]:
!unzip /content/gdrive/MyDrive/Networks/test_cases.zip -d /

In [ ]:
num_of_nodes = [20,25,30,35,40,45,50,55,60]
test_case_num = 30

#defining network parameters
GATEWAY_ID = 0
spread_factor = range(7,13)

In [ ]:
def read_data(path, isGraph):
  if isGraph:
    G = nx.read_gml(path,destringizer=int)
    return G
  with open(path,'rb') as file:
    data = pickle.load(file)
    file.close()
    return data


In [ ]:
def get_link_set(G,edge_nodes):
  all_paths = {}
  #get all possible paths to GATEWAY_ID for each flow
  for source in edge_nodes:
    all_paths[source] = []
    paths = nx.all_simple_paths(G,source,GATEWAY_ID)
    for path in paths:
      prev = path[0]
      path_edge_list = []
      for node in path[1:]:
        path_edge_list.append((prev,node))
        prev = node
      all_paths[source].append(path_edge_list)

  #generate link sets for each flow
  link_sets = {f : set([edge for path in all_paths[f] for edge in path]) for f in edge_nodes}
  return link_sets

In [ ]:
def gurobi_optimization(G, network_data, edge_nodes):
  #creating a list of bidirectional edges
  edge_list = []
  for u,v in G.edges:
    edge_list.append((u,v))
    edge_list.append((v,u))

  #get links sets for all the flows
  # link_sets = get_link_set(G, edge_nodes)

  N_DEVICES = len(G.nodes)

  #Setting the problem variable
  prob = Model(env = e)

  #setting the decision variables

  #y_f_i_j_k = f-flow (i,j) edge k spreading factor
  y = {(f,i,j,k):prob.addVar(lb = 0, ub = 1, vtype = GRB.BINARY, name = f'y_{f}_{i}_{j}_{k}') for f in edge_nodes for i,j in edge_list for k in spread_factor}

  #Defining the objective function
  prob.setObjective(quicksum(network_data['cst'][f,i,j,k]*y[f,i,j,k] for f in edge_nodes for i,j in edge_list for k in spread_factor), GRB.MINIMIZE)

  #adding flow conservation constraints
  prob.addConstrs(quicksum(y[f,i,j,k]-y[f,j,i,k] for j in G.adj[i] for k in spread_factor) == 0 for i in range(N_DEVICES) for f in edge_nodes if i not in {f,GATEWAY_ID})
  prob.addConstrs(quicksum(y[i,i,j,k]-y[i,j,i,k] for j in G.adj[i] for k in spread_factor) == 1 for i in edge_nodes)
  prob.addConstrs(quicksum(y[f,GATEWAY_ID,i,k]-y[f,i,GATEWAY_ID,k] for i in G.adj[GATEWAY_ID] for k in spread_factor) == -1 for f in edge_nodes)

  #adding link capacity constraints
  prob.addConstrs(y[f,i,j,k]*network_data['data_rate'][f,i,j,k] <= network_data['max_edge_data_rate'][k] for f in edge_nodes for i,j in edge_list for k in spread_factor)


  #adding delay constraints
  # prob.addConstrs(quicksum((network_data['edge_delays'][f,i,j] + network_data['tx_time'][k] + network_data['eh_time'][i,k])*y[f,i,j,k] for i,j in link_sets[f] for k in spread_factor) <= network_data['max_flow_delays'][f] for f in edge_nodes)
  prob.addConstrs(quicksum((network_data['edge_delays'][f,i,j] + network_data['tx_time'][k] + network_data['eh_time'][i,k])*y[f,i,j,k] for i,j in edge_list for k in spread_factor) <= network_data['max_flow_delays'][f] for f in edge_nodes)


  # prob.computeIIS()
  # prob.write("model.ilp")

  #solving the problem
  prob.optimize();
  # prob.write("out.lp")
  if prob.status not in (GRB.OPTIMAL, GRB.SUBOPTIMAL):
    return [-1,{}]

  optimal_cost = prob.objval
  optimal_path = {f : [(i,j,k) for i,j in edge_list for k in spread_factor if prob.getVarByName(f"y_{f}_{i}_{j}_{k}").x == 1] for f in edge_nodes}

  return [optimal_cost, optimal_path]

In [ ]:
expected_ans = {}

for N_DEVICES in num_of_nodes:
  expected_ans[N_DEVICES] = {}
  for t in range(1,test_case_num+1):

    #reading data
    network_data = {}
    G = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/graph.gml",True)
    network_data['tx_time'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/tx_time.txt", False)
    network_data['max_edge_data_rate'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/max_edge_data_rate.txt",False)
    network_data['cst'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/cst.txt",False)
    network_data['max_flow_delays'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/max_flow_delays.txt",False)
    network_data['edge_delays'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/edge_delays.txt",False)
    network_data['max_edge_delays'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/max_edge_delays.txt",False)
    network_data['data_rate'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/data_rate.txt",False)
    network_data['eh_time'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/eh_time.txt",False)
    network_data['init_res_energy'] = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/init_res_energy.txt",False)
    edge_nodes = read_data(f"test_cases/num_nodes_{N_DEVICES}/test_{t}/edge_nodes.txt",False)

    #get 2D coordinates of all the devices
    coordinates = {i: G.nodes()[i]['pos'] for i in G.nodes}

    # #visualizing the graph
    # print(f"Graph_{t+1}")
    # color_map = ["blue" for i in range(N_DEVICES)]
    # color_map[GATEWAY_ID] = "red"
    # plt.figure(figsize=(10,10))
    # nx.draw_networkx(G, coordinates,node_size = 300, node_color=color_map, font_color = "white", with_labels = True)
    # plt.show()

    optimal_cost, optimal_path = gurobi_optimization(G, network_data, edge_nodes)
    optimal_cost = round(optimal_cost, 2)
    expected_ans[N_DEVICES][t] = optimal_cost
    print(f"num_nodes = {N_DEVICES}, test_no = {t}, optimal_cost = {optimal_cost}")

    # for f in edge_nodes:
    #   print(optimal_path[f])

    # #visualizing the solution
    # flow_colors = dict(zip(edge_nodes,random.choices(list(mcolors.TABLEAU_COLORS),k = len(edge_nodes))))
    # node_color = ['black' for node in G.nodes()]
    # edge_color = ['black' for edge in G.edges()]
    # width = [1 for edge in G.edges()]
    # edge_labels = {}
    # for f in edge_nodes:
    #   for i,j,k in optimal_path[f]:
    #     node_color[i] = flow_colors[f]
    #     node_color[j] = flow_colors[f]
    #     if i < j:
    #       pos = list(G.edges()).index((i,j))
    #       edge_color[pos] = flow_colors[f]
    #       edge_labels[(i,j)] = f"SF-{k}"
    #       width[pos] = 3
    #     else:
    #       pos = list(G.edges()).index((j,i))
    #       edge_color[pos] = flow_colors[f]
    #       edge_labels[(j,i)] = f"SF-{k}"
    #       width[pos] = 3
    # node_color[0] = "red"

    # plt.figure(figsize=(10,10))
    # nx.draw_networkx(G, coordinates,
    #                 node_size = 350, 
    #                 node_color=node_color, 
    #                 width = width,
    #                 edge_color = edge_color, 
    #                 font_color = "white", 
    #                 font_weight = "bold", 
    #                 font_size = 14,
    #                 with_labels = True)
    # nx.draw_networkx_edge_labels(G, coordinates, edge_labels=edge_labels)
    # plt.show()

In [ ]:
#save expected results in a file
with open("/content/gdrive/MyDrive/Networks/expected_ans.pkl",'wb') as file:
  pickle.dump(expected_ans, file)
  file.close()